### You will learn to use Scikit-Learn, a machine learning library with Python, to predict whether a tweet is popular or not, from its image and text.

### 1. A small dataset
This dataset consists of (image, text) pairs of tweets, organized by the unique tweet ids. Check folder `./dataset/`. Each tweet has an id and a label: popular (1) or not popular (0), the information is saved in `./meta.csv`.

1.1 load  `meta.csv` using pandas to a dataframe, obtain a Nx2 numpy array M, first column is ID, second column is label
hint: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
 
1.2 split the rows of M randomly to 70% train, 20% validation, and 10% test: `M_train, M_val, M_test`
hint: you can use this [function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) twice.

1.3 Write a function `extract_feature_label(row)`, which takes a row of any meta matrix, read the image and text from its ID in `./dataset/`, and extract image features and text features. The function should return three things: label (0 or 1), image feature vector, text feature vector.

To read an image from file, you can use `skimage`, particularly this [function](https://scikit-image.org/docs/dev/api/skimage.io.html#imread)

You can use the provided functions `get_image_feature(image)`, `get_text_feature(text)` for help, but feel free to implement feature extraction on your own design. 

In [1]:
import numpy as np
import torch
from sklearn.feature_extraction.text import CountVectorizer
from skimage.transform import resize

# Load text corpus and prepare the Bigram.
corpus = open('./corpus.txt','r').readlines()
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
bigram_vectorizer.fit(corpus)

def get_image_feature(image, image_size=(128,128)):
    # image is an HxWxC numpy array.
    image = resize(image, (image_size[0], image_size[1]), anti_aliasing=True)
    print(image.shape)
    return image.reshape((-1))
    
def get_text_feature(text):
    # text is a String
    feature = bigram_vectorizer.transform([text]).toarray()
    return feature.reshape((-1))
    

In [6]:
def extract_feature_label(row):
    '''
    return label, image_feature, text_feature
    '''
    return

### 2. Prepare data in matrix form
Write a for loop to iterate over each of `M_train, M_val, M_test`. Use the above implemented function to aggregate the features and labels to numpy array forms. For example, for `M_train` with m entries, you should get `X_image_train (m x n), X_text_train (m x n), y_train (m x 1)`. Same applies for validation and test sets.

### 3. Learn to use scikit-learn to do classification, [tutorial](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In the above tutorial, pay attention to `examples` sample code. In your case, X should be a numpy array of m x n, where m is the number of tweets, n is the feature dimension, y should be a numpy array of m x 1, or a list, each element is either 1 or 0. Note the `score()` function gives the accuracy of prediction.

After learning this, you can call the `fit()` function on the training data, and `predict()` function on the test data.  You can use [this](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) to compute the accuracy of your prediction on the test data.

### 4. Now you are ready to do multimodal prediction!

4.1 Early fusion. 
For each tweet, concatenate its image feature and text feature to a new vector. That being said, you should concatenate the columns of `X_image_train` and `X_text_train` for example. The concatenated matrix is called `X_train`, then you can use Logistic regression!

4.2 Late fusion.
Use `X_image_train` and `X_text_train` separately to train two logistic regression classifiers `clf1, clf2`. 

Get two sets of prediction probabiities `y_image_prob_test` and `y_text_prob_test` using the two classifiers on the test set, use `predict_proba()` function in sklearn. These two should be two vectors with the same length. 

After getting the two probability vectors, compute the average probability vector `y_avg_prob_test` of these two. 

Use `y_avg_prob_test` to get a binary prediction vector `y_pred`. Use `y_pred `and `y_test` to compute accuracy.

### 5. Report

On the test set, report the following accuracies:
1. using image feature only
2. using text feature only
3. using early fusion
4. using late fusion

Extension:
1. try other classifiers in sklearn, e.g. KNN, decision tree
2. in late fusion, instead of compute average, you can also compute a weighted average, e.g. 30% for image, 70% for text if you think text is more important to predict popularity. Try different weights and see how the accuracy changes.